In [ ]:
import os
import random
import time
import itertools  
import numpy                 as np
import tensorflow            as tf
import matplotlib.pyplot     as plt
import pandas                as pd
import gudhi                 as gd
import gudhi.representations as sktda
import sys

from gudhi.tensorflow                     import LowerStarSimplexTreeLayer, CubicalLayer, RipsLayer
from gudhi.representations.vector_methods import Atol as atol
from gudhi.representations.kernel_methods import SlicedWassersteinKernel as swk
from gudhi.wasserstein                    import wasserstein_distance
from gudhi.representations                import pairwise_persistence_diagram_distances as ppdd
from mpl_toolkits.mplot3d                 import Axes3D
from scipy.linalg                         import expm
from scipy.io                             import loadmat
from scipy.sparse                         import csgraph
from scipy.linalg                         import eigh
from sklearn.base                         import BaseEstimator, TransformerMixin
from sklearn.metrics                      import pairwise_distances, accuracy_score
from sklearn.manifold                     import MDS, LocallyLinearEmbedding, SpectralEmbedding
from sklearn.preprocessing                import MinMaxScaler, Normalizer, LabelEncoder
from sklearn.pipeline                     import Pipeline, FeatureUnion
from sklearn.svm                          import SVC
from sklearn.ensemble                     import RandomForestClassifier
from sklearn.neighbors                    import KNeighborsClassifier
from sklearn.model_selection              import GridSearchCV, KFold, StratifiedKFold
from sklearn.cluster                      import KMeans

# Regression

In this section, we implement the linear regression experiment, where we recover hidden coefficients using 0-dimensional homology.

In [ ]:
np.random.seed(0)
n, p = 50, 100
betastar = np.concatenate([np.linspace(-1.,1.,33) for _ in range(3)] + [[-1.]])
X = np.random.multivariate_normal(mean=np.zeros(shape=[p]), cov=np.eye(p), size=n)

Y = np.matmul(X, betastar) + .05 * np.random.randn(n)
X, Y = np.array(X, dtype=np.float32), np.array(Y, dtype=np.float32)
stbase = gd.SimplexTree()
for i in range(p-1):
    stbase.insert([i,i+1], -1e10)

betainit = np.random.uniform(low=-1., high=1., size=[p])
betainit[np.array([25,60,99])] = np.array([-1,-1,-1])

In [ ]:
plt.figure()
plt.plot(betastar)
plt.title('Ground-truth coefficients')
plt.savefig('reggt.png')

In [ ]:
plt.figure()
plt.plot(betainit)
plt.title('Coefficients at epoch 0')
plt.savefig('reginit.png')

In [ ]:
beta = tf.Variable(initial_value=np.array(betainit[:,np.newaxis], dtype=np.float32), trainable=True)
layer = LowerStarSimplexTreeLayer(simplextree=stbase, dimension=0)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-5, decay_steps=10, decay_rate=.001)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms, betas = [], [], []
alpha, gamma, delta = 1, 1e4, 1e3
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        dgm = layer.call(beta)
        loss = alpha * tf.reduce_sum(tf.square(tf.matmul(X, -beta) - Y)) \
             + gamma * tf.reduce_sum(tf.abs(dgm[2:,1]-dgm[2:,0])) \
             + delta * tf.reduce_sum(tf.abs(beta[1:]-beta[:-1]))
             
    gradients = tape.gradient(loss, [beta])
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, [beta]))
    losses.append(loss.numpy())
    dgms.append(dgm)
    betas.append(beta.numpy()[:,0])

beta_stdtop = -betas[-1]

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('regloss.png')

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker='D', c='blue')
for dg in dgms[:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker='D', alpha=.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker='D', c='red')
plt.plot([-1,1], [-1,1])
plt.title('Optimized persistence diagrams')
plt.savefig('regdg.png')

In [ ]:
beta = tf.Variable(initial_value=np.array(betainit[:,np.newaxis], dtype=np.float32), trainable=True)
layer = LowerStarSimplexTreeLayer(simplextree=stbase, dimension=0)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-5, decay_steps=10, decay_rate=.001)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

In [ ]:
losses, dgms, betas = [], [], []
alpha, gamma, delta = 1, 0, 1e3
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        dgm = layer.call(beta)
        loss = alpha * tf.reduce_sum(tf.square(tf.matmul(X, -beta) - Y)) \
             + gamma * tf.reduce_sum(tf.abs(dgm[2:,1]-dgm[2:,0])) \
             + delta * tf.reduce_sum(tf.abs(beta[1:]-beta[:-1]))
             
    gradients = tape.gradient(loss, [beta])
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, [beta]))
    losses.append(loss.numpy())
    dgms.append(dgm)
    betas.append(beta.numpy()[:,0])
                 
beta_stdtot = -betas[-1]

beta = tf.Variable(initial_value=np.array(betainit[:,np.newaxis], dtype=np.float32), trainable=True)
layer = LowerStarSimplexTreeLayer(simplextree=stbase, dimension=0)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-5, decay_steps=10, decay_rate=.001)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms, betas = [], [], []
alpha, gamma, delta = 2, 0, 0
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        dgm = layer.call(beta)
        loss = alpha * tf.reduce_sum(tf.square(tf.matmul(X, -beta) - Y)) \
             + gamma * tf.reduce_sum(tf.abs(dgm[2:,1]-dgm[2:,0])) \
             + delta * tf.reduce_sum(tf.abs(beta[1:]-beta[:-1]))
                     
    gradients = tape.gradient(loss, [beta])
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, [beta]))
    losses.append(loss.numpy())
    dgms.append(dgm)
    betas.append(beta.numpy()[:,0])
    
beta_std = -betas[-1]

In [ ]:
plt.figure()
plt.plot(betastar, label='ground-truth')
plt.plot(beta_std, label='MSE')
plt.plot(beta_stdtot, label='MSE+TV')
plt.plot(beta_stdtop, label='MSE+TV+TOP')
plt.legend(loc='upper left')
plt.savefig('regafter.png')

In [ ]:
MSEstd, MSEtop, MSEtot = [], [], []
for s in range(1000):
    np.random.seed(s)
    Xnew = np.random.multivariate_normal(mean=np.zeros(shape=[p]), cov=np.eye(p), size=n)
    Ynew = np.matmul(Xnew, betastar)
    mse_std = np.square(np.matmul(Xnew, beta_std) - Ynew).sum()
    mse_tot = np.square(np.matmul(Xnew, beta_stdtot) - Ynew).sum()
    mse_top = np.square(np.matmul(Xnew, beta_stdtop) - Ynew).sum()
    MSEstd.append(mse_std)
    MSEtot.append(mse_tot)
    MSEtop.append(mse_top)
    
plt.figure()
plt.boxplot([MSEstd, MSEtot, MSEtop], labels=['MSE', 'MSE+TV', 'MSE+TV+TOP'])
plt.title('MSE on random test sets')
plt.savefig('regmse.png')

# Image

In this section, we implement the image experiment, where we remove the noise of an image using 0-dimensional homology. Use `use_reg=True` if you want to use a topological loss.

In [ ]:
use_reg = 1

In [ ]:
I = np.array(pd.read_csv('../difftda/data/mnist_test.csv', header=None, sep=','), dtype=np.float32)
idx = np.argwhere(I[:,0] == 8)
image = np.reshape(-I[idx[8],1:], [28,28])
image = (image-image.min())/(image.max()-image.min())
image_clean = np.array(image)
image[2:5,2:5]        -= 0.6
image[25:27,25:27]    -= 0.6
image[25:27,2:5]      -= 0.6
image[1:4,24:26]      -= 0.6

In [ ]:
plt.figure()
plt.imshow(image, cmap='Greys')
plt.title('Image at epoch 0')
plt.savefig('imbefore_' + str(use_reg) + '.png')

In [ ]:
X = tf.Variable(initial_value=np.array(image, dtype=np.float32), trainable=True)
layer = CubicalLayer(dimension=0)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-3, decay_steps=10, decay_rate=.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms, empty = [], [], np.empty([0,2])
alpha = 10.
gamma = 1. if use_reg else 0
for epoch in range(3000+1):
    
    with tf.GradientTape() as tape:
        
        dgm = layer.call(X)
        if use_reg:
            loss = alpha * tf.math.reduce_sum(tf.abs(dgm[:,1]-dgm[:,0])) + \
                   gamma * tf.math.reduce_sum(tf.math.minimum(tf.abs(X), tf.abs(1-X)))
        else:
            loss = alpha * tf.math.reduce_sum(tf.abs(dgm[:,1]-dgm[:,0]))

    gradients = tape.gradient(loss, [X])
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, [X]))
    losses.append(loss.numpy())
    dgms.append(dgm)

In [ ]:
plt.figure()
plt.imshow(X.numpy(), cmap='Greys')
plt.title('Image at epoch ' + str(epoch))
plt.savefig('imafter_' + str(use_reg) + '.png')

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('imloss_' + str(use_reg) + '.png')

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker='D', c='blue')
for dg in dgms[:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker='D', alpha=0.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker='D', c='red')
plt.plot([0,1], [0,1])
plt.title('Optimized persistence diagrams')
plt.savefig('imdg_' + str(use_reg) + '.png')

# Point cloud

In this section, we implement the point cloud experiment, where we optimize loops in a point cloud using 1-dimensional homology.

In [ ]:
use_reg = 1

In [ ]:
np.random.seed(1)
Xinit = np.array(np.random.uniform(high=1., low=-1., size=(300,2)), dtype=np.float32)

In [ ]:
plt.figure()
plt.scatter(Xinit[:,0], Xinit[:,1])
plt.title('Point cloud at epoch 0')
plt.savefig('pcinit_' + str(use_reg) + '.png')

In [ ]:
X = tf.Variable(initial_value=Xinit, trainable=True)
layer = RipsLayer(maximum_edge_length=12., dimension=1)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-1, decay_steps=10, decay_rate=.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.)
sigma = 0.001

losses, dgms = [], []
for epoch in range(1000+1):
    
    with tf.GradientTape() as tape:
        
        dgm = layer.call(X)
        if use_reg:
            loss = -tf.math.reduce_sum(tf.square(.5*(dgm[:,1]-dgm[:,0]))) + tf.reduce_sum(tf.maximum(tf.abs(X)-1, 0))
        else:
            loss = -tf.math.reduce_sum(tf.square(.5*(dgm[:,1]-dgm[:,0])))

    gradients = tape.gradient(loss, [X])
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, [X]))
    losses.append(loss.numpy())
    dgms.append(dgm)

In [ ]:
plt.figure()
plt.scatter(X.numpy()[:,0], X.numpy()[:,1])
plt.title('Point cloud at epoch ' + str(epoch))
plt.savefig('pcafter_' + str(use_reg) + '.png')

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('pcloss_' + str(use_reg) + '.png')

In [ ]:
plt.figure()
for dg in dgms[:5:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker='D', alpha=0.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker='D', c='green')
plt.plot([-0.,.2], [-0.,.2])
plt.title('Optimized persistence diagrams')
plt.savefig('pcdg_' + str(use_reg) + '.png')

# Noisy point cloud

In this section, we implement the noisy point cloud experiment, where we optimize the connected components of a noisy point cloud using 0-dimensional homology.

In [ ]:
n, epsilon, nout = 100, .2, 3
x, y = np.cos(np.linspace(0,2*np.pi,n)), np.sin(np.linspace(0,2*np.pi,n))
np.random.seed(10)
ex, ey = np.random.uniform(low=-epsilon,high=epsilon,size=n), np.random.uniform(low=-epsilon,high=epsilon,size=n)
outliers = np.random.uniform(low=-.7, high=.7, size=(nout,2))

In [ ]:
plt.figure()
plt.scatter(x+ex, y+ey)
plt.scatter(outliers[:,0], outliers[:,1])
plt.title('Point cloud at epoch 0')
plt.savefig('noisypcinit.png')

In [ ]:
st = gd.RipsComplex(distance_matrix=pairwise_distances(np.hstack([x[:,np.newaxis],y[:,np.newaxis]])), max_edge_length=2.).create_simplex_tree(max_dimension=2)
st.persistence()
D = np.array(st.persistence_intervals_in_dimension(0), dtype=np.float32)[:-1]

In [ ]:
Xinit = np.array(np.vstack([np.hstack([(x+ex)[:,np.newaxis], (y+ey)[:,np.newaxis]]),outliers]), dtype=np.float32)

X = tf.Variable(initial_value=Xinit, trainable=True)
layer = RipsLayer(maximum_edge_length=2., dimension=0)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-1, decay_steps=10, decay_rate=.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms = [], []
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        dgm = layer.call(X)
        loss = tf.square(wasserstein_distance(dgm, tf.constant(D), order=2, enable_autodiff=True))
        
    gradients = tape.gradient(loss, [X])
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, [X]))
    losses.append(loss.numpy())
    dgms.append(dgm)        

In [ ]:
plt.figure()
plt.scatter(X.numpy()[:,0], X.numpy()[:,1])
plt.title('Point cloud at epoch ' + str(epoch))
plt.savefig('noisypcafter.png')

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('noisypcloss.png')

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker='D', c='blue')
for dg in dgms[:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker='D', alpha=.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker='D', c='red')
plt.plot([-1,1], [-1,1])
plt.title('Optimized persistence diagrams')
plt.savefig('noisypcdg.png')

# 3D shape

In this section, we implement the 3D shape experiment, where we optimize the values on a 3D shape using 0-dimensional homology.

In [ ]:
faces, coord = np.loadtxt('../difftda/data/human_faces', dtype=float)[:,1:], np.loadtxt('../difftda/data/human_coords', dtype=float)
stbase = gd.SimplexTree()
for i in range(len(faces)):
    stbase.insert(faces[i,:], -1e10)
Finit = coord[:,2]

In [ ]:
step = 1
fig = plt.figure()
cm = plt.cm.get_cmap('rainbow')
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(coord[::step,0], coord[::step,1], coord[::step,2], c=Finit[::step], s=2, 
                vmin=0, vmax=.75, cmap=cm)
x_limits, y_limits, z_limits = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
x_range, x_middle = abs(x_limits[1] - x_limits[0]), np.mean(x_limits)
y_range, y_middle = abs(y_limits[1] - y_limits[0]), np.mean(y_limits)
z_range, z_middle = abs(z_limits[1] - z_limits[0]), np.mean(z_limits)
plot_radius = 0.5*max([x_range, y_range, z_range])
ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
plt.title('Function at epoch 0')
plt.savefig('d3sinit.png')

In [ ]:
F = tf.Variable(initial_value=np.array(Finit, dtype=np.float32), trainable=True)
layer = LowerStarSimplexTreeLayer(simplextree=stbase, dimension=0)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-1, decay_steps=10, decay_rate=.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms = [], []
alpha, gamma = 1., .001
for epoch in range(3000+1):
    
    with tf.GradientTape() as tape:
        
        dgm = layer.call(F)
        loss = alpha * tf.square(wasserstein_distance(dgm, tf.constant(np.array([[-.98,-.03]], dtype=np.float32)), order=2, enable_autodiff=True))               
    
    gradients = tape.gradient(loss, [F])
    np.random.seed(epoch)
    gradients = [tf.convert_to_tensor(gradients[0]) + np.random.normal(loc=0., scale=sigma, size=gradients[0].dense_shape)]
    optimizer.apply_gradients(zip(gradients, [F]))
    losses.append(loss.numpy())
    dgms.append(dgm)

In [ ]:
step = 1
fig = plt.figure()
cm = plt.cm.get_cmap('rainbow')
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(coord[::step,0], coord[::step,1], coord[::step,2], c=F.numpy()[::step], s=1, 
                vmin=0, vmax=.75, cmap=cm)
x_limits, y_limits, z_limits = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
x_range, x_middle = abs(x_limits[1] - x_limits[0]), np.mean(x_limits)
y_range, y_middle = abs(y_limits[1] - y_limits[0]), np.mean(y_limits)
z_range, z_middle = abs(z_limits[1] - z_limits[0]), np.mean(z_limits)
plot_radius = 0.5*max([x_range, y_range, z_range])
ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
plt.title('Function at epoch ' + str(epoch))
plt.savefig('d3safter.png')

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('d3sloss.png')

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker='D', c='blue')
for dg in dgms[:-1:2]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker='D', alpha=.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker='D', c='red')
plt.plot([-1,1], [-1,1])
plt.title('Optimized persistence diagrams')
plt.savefig('d3sdg.png')

# Filter selection

In this section, we implement the filter selection experiment, where we optimize a filter for classification using 0-dimensional homology.

In [ ]:
class FiltrationSelector(BaseEstimator, TransformerMixin):

    def __init__(self, use=False, index_filt=0):
        self.use, self.index_filt = use, index_filt

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.use:
            Xfit = [D[self.index_filt] for D in X]
        else:
            Xfit = X
        return Xfit

In [ ]:
dataset                   = '23'
step                      = 10
initial_learning_rate     = 0.001
batch_size                = 100
num_epochs                = 20
numdir                    = 10
hcard                     = 20
hdim                      = 0

In [ ]:
thetainit = np.linspace(-np.pi/2, np.pi/2, num=numdir)
Cinit = np.array([0., np.pi/2])
num_filts = len(Cinit)
X = tf.keras.datasets.mnist.load_data()

In [ ]:
l1, l2 = int(dataset[0]), int(dataset[1])
tridxs1, tridxs2 = np.argwhere(X[0][1] == l1).ravel()[::step], np.argwhere(X[0][1] == l2).ravel()[::step]
teidxs1, teidxs2 = np.argwhere(X[1][1] == l1).ravel()[::step], np.argwhere(X[1][1] == l2).ravel()[::step]
IMG = [X[0][0][j] for j in tridxs1]    + [X[0][0][j] for j in tridxs2]    + [X[1][0][j] for j in teidxs1]    + [X[1][0][j] for j in teidxs2]
LAB = [0 for _ in range(len(tridxs1))] + [1 for _ in range(len(tridxs2))] + [0 for _ in range(len(teidxs1))] + [1 for _ in range(len(teidxs2))]
ntrain = len(tridxs1) + len(tridxs2)
ntot = len(tridxs1) + len(tridxs2) + len(teidxs1) + len(teidxs2)
train_idxs, test_idxs = np.arange(0, ntrain), np.arange(ntrain, ntot)

In [ ]:
DGMb = []
for pdi in range(num_filts):
    DGMi = []
    for j, img in enumerate(IMG):
        inds = np.argwhere(img > 0)
        I = np.inf * np.ones(img.shape)
        for i in range(len(inds)):
            val = np.cos(Cinit[pdi])*inds[i,0] + np.sin(Cinit[pdi])*inds[i,1]
            I[inds[i,0], inds[i,1]] = val
        ccb = gd.CubicalComplex(top_dimensional_cells=I)
        ccb.persistence()
        dgmb = ccb.persistence_intervals_in_dimension(hdim)
        DGMi.append(dgmb)

        if j == 0:
            vm, vM = min(list(I[I!=np.inf].flatten())), max(list(I[I!=np.inf].flatten()))
            plt.figure()
            plt.imshow(I, vmin=vm, vmax=vM)
            plt.colorbar()
            plt.savefig(str(j) + '_' + '{:.2f}'.format(Cinit[pdi]) + '.png')

    DGMb.append(DGMi)

In [ ]:
num_filts = len(DGMb)
num_diags = len(DGMb[0])
DGMb = [[DGMb[f][i] for f in range(num_filts)] for i in range(num_diags)]

train_dgmbs, test_dgmbs = [DGMb[i] for i in train_idxs], [DGMb[i] for i in test_idxs]
train_labs,  test_labs  = [LAB[i]  for i in train_idxs], [LAB[i]  for i in test_idxs]
le = LabelEncoder().fit(train_labs + test_labs)
train_labs, test_labs = le.transform(train_labs), le.transform(test_labs)

pipe = Pipeline([
    ('Feats', FeatureUnion([  ('Pipe' + str(nf), Pipeline([('Selector',  FiltrationSelector(index_filt=nf)),
                                                           ('Separator', sktda.DiagramSelector(limit=np.inf, point_type='finite')),
                                                           ('TDA',       sktda.Landscape())
                                                          ])) for nf in range(num_filts)
                           ])),
    ('Estimator', RandomForestClassifier())
])
param = {'Feats__Pipe0__Selector__use':        True,
         'Feats__Pipe0__Separator__use':       True,
         'Feats__Pipe0__TDA__resolution':      50,
         'Feats__Pipe0__TDA__num_landscapes':  5}
for nf in range(num_filts-1):
    newparam = {'Feats__Pipe' + str(nf+1) + '__Selector__use':        True,
                'Feats__Pipe' + str(nf+1) + '__Separator__use':       True,
                'Feats__Pipe' + str(nf+1) + '__TDA__resolution':      50,
                'Feats__Pipe' + str(nf+1) + '__TDA__num_landscapes':  5}
    param.update(newparam)
param['Estimator__random_state'] = 0

modelb = pipe.set_params(**param)
modelb.fit(train_dgmbs, train_labs)
trb = modelb.score(train_dgmbs, train_labs)
teb = modelb.score(test_dgmbs,  test_labs)
train_imgs = np.vstack([IMG[i].flatten()[np.newaxis,:] for i in train_idxs]) 
test_imgs  = np.vstack([IMG[i].flatten()[np.newaxis,:] for i in test_idxs])
rf = RandomForestClassifier().fit(train_imgs, train_labs)
trbb = rf.score(train_imgs, train_labs)
tebb = rf.score(test_imgs,  test_labs)

In [ ]:
C = tf.Variable(initial_value=np.array(Cinit, dtype=np.float32), trainable=True)
thetas = tf.Variable(initial_value=np.array(thetainit, dtype=np.float32), trainable=False)
lr = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=1e5, decay_rate=0.99, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
batch_size = min(batch_size, len(train_idxs))

losses, coeffs, accs = [], [], []
for epoch in range(num_epochs+1):
    
    np.random.seed(int(1e2*epoch))
    batch = np.random.choice(train_idxs, batch_size, replace=False)
    batch_labs = [LAB[i] for i in batch]
    
    with tf.GradientTape() as tape:

        dists = []
        for nf in range(num_filts):
            
            dgms = []
            for i in batch:
                img = IMG[i]
                inds = np.argwhere(img > 0)
                IX, IY = 1e3 * np.ones(img.shape), 1e3 * np.ones(img.shape)
                for k in range(len(inds)):
                    IX[inds[k,0], inds[k,1]] = inds[k,0]
                    IY[inds[k,0], inds[k,1]] = inds[k,1]
                II = tf.math.cos(C[nf])*IX + tf.math.sin(C[nf])*IY
                dgm = CubicalLayer(dimension=hdim).call(II)
                dgms.append(dgm)
        
            dgms = [tf.pad(d,tf.constant([[0,hcard-d.shape[0]],[0,0]])) for d in dgms]
            proj_dgms = tf.linalg.matmul(tf.concat(dgms,axis=0), .5*tf.ones([2,2], tf.float32))
            dgms_big = tf.concat([tf.reshape(tf.concat([
                dgm, proj_dgms[:hcard*idg], proj_dgms[hcard*(idg+1):]
            ], axis=0), [-1,2,1,1]) for idg, dgm in enumerate(dgms)], axis=2)
            cosines, sines = tf.math.cos(thetas), tf.math.sin(thetas)
            vecs = tf.concat([tf.reshape(cosines,[1,1,1,-1]), tf.reshape(sines,[1,1,1,-1])], axis=1)
            theta_projs = tf.sort(tf.math.reduce_sum(tf.math.multiply(dgms_big,vecs), axis=1), axis=0)
            t1 = tf.reshape(theta_projs, [hcard*batch_size,-1,1,numdir])
            t2 = tf.reshape(theta_projs, [hcard*batch_size,1,-1,numdir])
            dists.append(tf.math.reduce_mean(tf.math.reduce_sum(tf.math.abs(t1-t2), axis=0), axis=2))

        loss = 0.
        classes = np.unique(batch_labs)
        for l in classes:
            lidxs = np.argwhere(np.array(batch_labs) == l).ravel()
            idxs1 = list(itertools.product(lidxs, lidxs))
            idxs2 = list(itertools.product(lidxs, range(batch_size)))
            for nf in range(num_filts):
                cost1 = tf.math.reduce_sum(tf.gather_nd(dists[nf], idxs1))
                cost2 = tf.math.reduce_sum(tf.gather_nd(dists[nf], idxs2))
                loss += cost1 / cost2
    
    gradients = tape.gradient(loss, [C]) 
    optimizer.apply_gradients(zip(gradients, [C]))
    curr_coeff = C.numpy().flatten()
    losses.append(loss.numpy())
    coeffs.append(curr_coeff)
    if epoch % 10 == 0:

        final_coeff = C.numpy()

        DGM = []
        for nf in range(num_filts):
            DGMi = []
            for i, img in enumerate(IMG):
                inds = np.argwhere(img > 0)
                I = np.inf * np.ones(img.shape)
                for i in range(len(inds)):
                    val = np.cos(final_coeff[nf])*inds[i,0] + np.sin(final_coeff[nf])*inds[i,1]
                    I[inds[i,0], inds[i,1]] = val
                cc = gd.CubicalComplex(top_dimensional_cells=I)
                cc.persistence()
                dgm = cc.persistence_intervals_in_dimension(hdim)
                DGMi.append(dgm)
            DGM.append(DGMi)

        num_filts = len(DGM)
        num_diags = len(DGM[0])
        DGM = [[DGM[f][i] for f in range(num_filts)] for i in range(num_diags)]
        train_dgms, test_dgms = [DGM[i] for i in train_idxs], [DGM[i] for i in test_idxs]
        model = pipe.set_params(**param)
        model.fit(train_dgms, train_labs)
        tr = model.score(train_dgms, train_labs)
        te = model.score(test_dgms,  test_labs)

        accs.append([trbb, tebb, trb, teb, tr, te])

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('fltloss.png')

In [ ]:
plt.figure()
plt.plot([accs[i][0] for i in range(len(accs))], label='baseline-tr')
plt.plot([accs[i][1] for i in range(len(accs))], label='baseline-te')
plt.plot([accs[i][2] for i in range(len(accs))], label='w/o optim-tr')
plt.plot([accs[i][3] for i in range(len(accs))], label='w/o optim-te')
plt.plot([accs[i][4] for i in range(len(accs))], label='optim-tr')
plt.plot([accs[i][5] for i in range(len(accs))], label='optim-te')
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.legend()
plt.savefig('fltacc.png')

In [ ]:
CS = np.array(coeffs)
plt.figure()
for c in range(CS.shape[1]):
    plt.plot(CS[:,c])
plt.savefig('fltcoeff.png')